**1. Normalizasyon**
   Veri setinizi normalleştirmek, farklı ölçeklerde olan özelliklerin modeller tarafından daha etkili bir şekilde işlenmesini sağlar.En yaygın normalleştirme yöntemlerinden biri Min-Max normalleştirmesidir. Bu yöntem, tüm değerleri 0 ile 1 arasında bir ölçeğe dönüştürür.
   
Değişken içinde yer alan sayıları genellikle 0 ve 1 arasına hapseden bir yöntemdir. Min-Max Scalling olarak da bilinir.

Xnorm = (X − Xmin) / (Xmax − Xmin)

Burada, Xmax ve Xmin sırasıyla değişkene ait değerlerin maksimum ve minimum sayılarını verir.

X değeri değişkendeki minimum değer olduğunda, pay 0 olacaktır ve dolayısıyla X = 0 olacaktır.
Öte yandan, X’in değeri değişkendeki maksimum değer olduğunda, pay paydaya eşittir olacağından X = 1 sonucu çıkacaktır.
Dolayısıyla X’in değeri minimum ve maksimum değer arasındaysa, X’in değeri 0 ile 1 arasında anlamına gelir.

In [21]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
df = pd.read_csv('veri-seti.txt', header=None, sep='\t')
columns = ['Number of times pregnant', 'Plasma glucose', 'Diastolic blood pressure', 'Triceps skinfold thickness ', '2-Hour serum insulin', 'Body mass index', 'Diabetes pedigree function.', 'Age', 'Outcome']
df.columns = columns
df_normalized = (df - df.min()) / (df.max() - df.min())

# İlk beş satırı kontrol edelim
df_normalized.head()


,Number of times pregnant,Plasma glucose,Diastolic blood pressure,Triceps skinfold thickness,2-Hour serum insulin,Body mass index,Diabetes pedigree function.,Age,Outcome
0,0.352941,0.743719,0.590164,0.353535,0.000000,0.500745,0.234415,0.483333,1.0
1,0.058824,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,0.166667,0.0
2,0.470588,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,0.183333,1.0
3,0.058824,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,0.000000,0.0
4,0.000000,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,0.200000,1.0


**2. PCA ve LDA Uygulaması**
PCA (Principal Component Analysis) ve LDA (Linear Discriminant Analysis) boyut indirgeme tekniğidir. PCA, verinin varyansını maksimize ederek özellikleri dönüştürürken, LDA sınıf ayrımını maksimize eder. Her iki yöntem de genellikle yüksek boyutlu veri setlerini daha az boyutlu bir uzaya indirgemek için kullanılır. Ancak, LDA denetimli bir öğrenme yöntemidir ve sınıf etiketlerini kullanır, PCA ise denetimsiz bir yöntemdir.

In [23]:
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split

X = df_normalized.drop('Outcome', axis=1)
y = df_normalized['Outcome']

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

lda = LDA(n_components=1) 
X_lda = lda.fit_transform(X, y)

print("PCA Varyans Oranı:", pca.explained_variance_ratio_)
print("LDA Varyans Oranı:", lda.explained_variance_ratio_ if hasattr(lda, 'explained_variance_ratio_') else "N/A")


PCA Varyans Oranı: [0.31192249 0.21186663]
LDA Varyans Oranı: [1.]


**3. Çoklu Doğrusal ve Multinominal Lojistjik Regresyon**

In [ ]:
iki veya daha fazla bağımsız değişkeni bir bağımlı değişkenle ilişkilendiren bir denklem bulmak için çoklu doğrusal regresyon modeli kullanılır.
Multinominal Lojistik Regresyon analizi, bağımlı değişkenin iki veya daha fazla kategorik sınıfa ait olabileceği durumlarda kullanılır.

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

log_reg = LogisticRegression(multi_class='multinomial', solver='lbfgs')
log_reg.fit(X_train, y_train)
print("Katsayılar:\n", log_reg.coef_)
print("Kesme terimi:", log_reg.intercept_)

# Test verisi üzerinde performans değerlendirme
y_pred = log_reg.predict(X_test)
print("Doğruluk oranı:", accuracy_score(y_test, y_pred))
print("Sınıflandırma raporu:\n", classification_report(y_test, y_pred))


Katsayılar:
 [[ 0.4505206   2.65118931 -0.24999553  0.03039025 -0.06304149  2.16599472
   0.45399225  0.90638614]]
Kesme terimi: [-3.25065341]
Doğruluk oranı: 0.7359307359307359
Sınıflandırma raporu:
               precision    recall  f1-score   support

         0.0       0.78      0.82      0.80       151
         1.0       0.63      0.57      0.60        80

    accuracy                           0.74       231
   macro avg       0.71      0.70      0.70       231
weighted avg       0.73      0.74      0.73       231



Bu sonuçlar, modelin diyabet teşhisi konusunda iyi bir doğruluk oranına sahip olduğunu, ancak özellikle pozitif sınıf için (diyabet olanlar) duyarlılık ve kesinliğin daha da iyileştirilebileceğini göstermektedir. Yanlış negatif ve yanlış pozitif oranlarının azaltılması için modelin daha da optimize edilmesi gerekebilir.

**4. Karar Ağacı Sınıflandırma**

In [ ]:
Karar ağacı sınıflandırma, karmaşık veri setlerini basit karar kurallarına dönüştürerek yorumlanabilirlik sağlar.Anlaşılabilir bir yapıya sahip olduğu için sonuçların nedenleri kolayca anlaşılabilir

In [26]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train, y_train)

y_pred_dt = decision_tree.predict(X_test)
accuracy_dt = accuracy_score(y_test, y_pred_dt)
conf_matrix_dt = confusion_matrix(y_test, y_pred_dt)
class_report_dt = classification_report(y_test, y_pred_dt)

print("Doğruluk (Accuracy):", accuracy_dt)
print("Confusion Matrix:\n", conf_matrix_dt)
print("Sınıflandırma Raporu:\n", class_report_dt)


Doğruluk (Accuracy): 0.7012987012987013
Confusion Matrix:
 [[107  44]
 [ 25  55]]
Sınıflandırma Raporu:
               precision    recall  f1-score   support

         0.0       0.81      0.71      0.76       151
         1.0       0.56      0.69      0.61        80

    accuracy                           0.70       231
   macro avg       0.68      0.70      0.69       231
weighted avg       0.72      0.70      0.71       231



Karar Ağacı modeli, diyabet teşhisi konusunda kabul edilebilir bir doğruluk oranı sunmuş, ancak yanlış pozitif ve yanlış negatif oranları dikkate alındığında modelin performansının daha da iyileştirilebileceği görülmektedir. Diyabet olmayanları tespit etme konusunda daha yüksek bir kesinlik sunarken, diyabet olan hastaların tespitinde duyarlılığı artırılabilir.

**5. Naive Bayes Sınıflandırma**

In [25]:
from sklearn.naive_bayes import GaussianNB

naive_bayes = GaussianNB()
naive_bayes.fit(X_train, y_train)

y_pred_nb = naive_bayes.predict(X_test)
print("Oran:", accuracy_score(y_test, y_pred_nb))
print("Conf. Matrix:\n", confusion_matrix(y_test, y_pred_nb))
print("Sınıflandırma:\n", classification_report(y_test, y_pred_nb))


Oran: 0.7445887445887446
Conf. Matrix:
 [[119  32]
 [ 27  53]]
Sınıflandırma:
               precision    recall  f1-score   support

         0.0       0.82      0.79      0.80       151
         1.0       0.62      0.66      0.64        80

    accuracy                           0.74       231
   macro avg       0.72      0.73      0.72       231
weighted avg       0.75      0.74      0.75       231



Naive Bayes modeli, test verisi üzerinde %74.46 gibi bir doğruluk oranı ile kabul edilebilir bir performans göstermiştir.Bu sonuçlar, modelin iyileştirilmesi için ek parametre ayarlamaları veya farklı sınıflandırma yöntemlerinin denenmesi gerektiğini göstermektedir.